# Creating an Agent via boto3 SDK
Auther: xiaoqunn@amazom.com (GCR ALML GenAI SSA)

Creating an Agent via boto3 SDK
* Step 0. Pre-requests
  + Prepare python environments.
  + Open AWS SES service: https://us-east-1.console.aws.amazon.com/ses/home?region=us-east-1#/get-set-up
* Step 1. Create Lambda function & upload relevant files to s3
  + Step 1.1 Create lambda roles
  + Step 1.2 Create lambda function
  + Step 1.3 Upload relavent files
  + Step 1.4 Create lambda layer & add layer
  + Step 1.5 Create bedrock agent role
* Step 2. Create an Agent
* Step 3. Create Action group
* Step 4. Associate knowledge base(Optional)
* Step 5. Prepare agent and create agent alias
* Step 6. Invoking Agent
* Test

### Step 0. Pre-requests.

#### Prepare python environments.

In [ ]:
!pip install boto3

#### Open AWS SES service

url: https://us-east-1.console.aws.amazon.com/ses/home?region=us-east-1#/get-set-up

In [2]:
%%HTML
<img src="../imgs/Picture1.png", width=600>

After you verify eamil, you need to modify invoice_lambda.py <br>
<br>
<strong>invoice_lambda.py -> send_eamil function -> SENDER = "Your verified email address."</strong>

### Step 1. Create Lambda function & upload relevant files

In [5]:
import boto3
import uuid
import logging
import json

import os
from botocore.exceptions import ClientError


logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
aws_access_key_id = "your_access_key_id"
aws_secret_access_key = "your_secret_access_key"
region = "us-east-1"
iam = boto3.resource("iam", region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
lambda_client= boto3.client("lambda", region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)



#### Step 1.1 Create lambda roles

In [ ]:
# create lambda role for bedrock agent
from utils import create_role, create_policy, attach_policy, get_role
lambda_agent_demo_role = create_role(
    iam,
    "bedrock-agent-lambda-demo",
    ["lambda.amazonaws.com", "bedrock.amazonaws.com"]
    )
print(f"{lambda_agent_demo_role.name}, {lambda_agent_demo_role.arn}")

In [ ]:
# attach full access policy
full_access_lambda_policy = create_policy(
    iam,
    "full-demo-policy",
    "Full access for agent lambda demo.",
    "*",
    "*"
)
attach_policy(
    iam,
    "bedrock-agent-lambda-demo",
    full_access_lambda_policy.arn
)

#### Step 1.2 Create lambda function

In [8]:
from utils import create_function, create_deployment_package

In [ ]:
# Create lambda function
bedrock_agent_lambda_demo = get_role(iam, "bedrock-agent-lambda-demo")
print("Zipping the Python script into a deployment package...")

lambda_name = "invoice_test"
deployment_package = create_deployment_package(
    "../invoice_lambda.py",
    f"{lambda_name}.py"
)
print(f"...and creating the {lambda_name} Lambda function.")

lambda_function_arn = create_function(
    lambda_client,
    lambda_name,
    f"{lambda_name}.lambda_handler",
    lambda_agent_demo_role,
    deployment_package
)
print(f"Successfully create the {lambda_name} Lambda function.")

#### Step 1.3 Upload relavent files

In [ ]:
import os
import random
bucket_name = "invoice-agent-demo" + str(random.randint(1,20))
s3_client = boto3.client("s3", region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Upload demo relavant files
file_path = "../conf"
file_list = os.listdir(file_path)
print(file_list)
response = s3_client.create_bucket(Bucket=bucket_name)
for file_name in file_list:
    upload_schema = s3_client.upload_file(os.path.join(file_path, file_name), bucket_name, file_name)


# Upload lambda layer
lambda_layer = "invoice_lambda_layer.zip"
respose = s3_client.upload_file("../"+lambda_layer, bucket_name, lambda_layer)
print(response)

# Upload API schema file
invoice_schema = "invoic_schema.json"
respose = s3_client.upload_file("../conf/invoice_service_schema.json", bucket_name, invoice_schema)
print(response)
s3_schema_path = f"arn:aws:s3:::{bucket_name}/{invoice_schema}"

#### Step 1.4 Create lambda layer & add layer

How to create a Python layer for your dependencies, reference link: https://docs.aws.amazon.com/lambda/latest/dg/python-package.html

In [ ]:
LayerName = "invoice-demo-layer"
response = lambda_client.publish_layer_version(
    LayerName=LayerName,
    Description='an layer for agent lambda function',
    Content={
        'S3Bucket': bucket_name,
        'S3Key': lambda_layer,
    },
    CompatibleRuntimes=[
        'python3.10',
    ],
    CompatibleArchitectures=[
        'x86_64',
    ]
)

layer_arn = response["LayerArn"]
print(f"layer_arn: {layer_arn}")

In [ ]:
# Add BUCKET_NAME parameter and lambda layer
lambda_client.update_function_configuration(
    FunctionName=lambda_name, Environment={"Variables": {"BUCKET_NAME": bucket_name}}, Layers=[layer_arn+":1"]
)

In [23]:
#Add permission

account_id = "004550504873" # Your Account ID
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allowinvoke',
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=f"arn:aws:bedrock:us-east-1:{account_id}:agent/*", 
    SourceAccount=account_id
)

In [ ]:
print(response)

#### Step 1.5 Create bedrock agent role

In [ ]:
# Create Bedrock Agent role
# Role name must startwith "AmazonBedrockExecutionRoleForAgents_" 
bedrock_agent_role = create_role(
        iam,
        "AmazonBedrockExecutionRoleForAgents_demo",
        ["bedrock.amazonaws.com"]
    )
print(bedrock_agent_role.arn)

In [ ]:
# Create s3 policy and bedrock invoke policy
s3_schema_policy = create_policy(
    iam,
    "invoice-schema-policy",
    "Policy for IAM demonstration.",
    ["s3:GetObject"], 
    s3_schema_path
)
bedrock_agent_invoke_demo_policy = create_policy(
    iam,
    "invoice-bedrock-agent-demo-policy",
    "Policy for IAM demonstration.",
    "bedrock:InvokeModel",
    "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2"
)

# If you want to add an knowledge base, Uncomment it

# knowledge_base_arn = f"arn:aws:bedrock:us-east-1:{account_id}:knowledge-base/*"

# kb_retrive_policy = create_policy(
#     iam,
#     "invoice-agent-kb-demo-policy",
#     "Policy for agent kb retreive.",
#     ["bedrock:Retrieve"],
#     [knowledge_base_arn] 
# )

In [ ]:
# attach policy to Bedrock Agent role
attach_policy(
    iam,
    "AmazonBedrockExecutionRoleForAgents_demo",
    f"arn:aws:iam::{account_id}:policy/invoice-bedrock-agent-demo-policy"
)
attach_policy(
    iam,
    "AmazonBedrockExecutionRoleForAgents_demo",
    f"arn:aws:iam::{account_id}:policy/invoice-schema-policy"
)

# for knowledge base retrieve, uncomment it 
# attach_policy(
#     iam,
#     "AmazonBedrockExecutionRoleForAgents_demo",
#     kb_retrive_policy.arn 
# )

### Step 2. Create an Agent

In [54]:
client = boto3.client("bedrock-agent", region_name=region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

agent_name = "invoice-test-demo"
instructions =("You are a friendly Invoice assistant. When greeted, use a greeting term and "
               "\"I'm an invoice assistant\" to answer. Through the \"InvoiceService\" action group, you can offer invoice services. "
               "When generating an invoice, first collect all required invoice information from user. "
               "Then generate a temporary preview image for the user's reference. And return the text_info. "
               "Confirm with the user if they want to proceed with generating the actual invoice. If user confirms, use tools to generate the invoice. "
               "If successful, return the downloadUrl from the function result to user. This allows user to download the invoice. "
               "If user indicates the information is incorrect, ask them to provide corrected information. Confirm if user needs the invoice sent to a designated email address. "
               "If so, email the invoice file to the address provided.")

response = client.create_agent(
    agentName = agent_name,
    agentResourceRoleArn = bedrock_agent_role.arn,
    description = "invocie test",
    idleSessionTTLInSeconds = 1800,
    foundationModel = "anthropic.claude-v2",
    instruction = instructions,
)
agent_id = response['agent']['agentId']

### Step 3. Create Action group

In [55]:
action_group_name = "InvoiceService"
s3_bucket = bucket_name
s3_object_key = invoice_schema
description = "An invocie service"

response = client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT', 
    actionGroupExecutor={
        'lambda': lambda_function_arn
        },
    actionGroupName = action_group_name,
    apiSchema={
        's3': {
            's3BucketName': s3_bucket,
            's3ObjectKey': s3_object_key
            }
    },
    description = description
)

### Step 4. Associate knowledge base(Optional)

In [ ]:
# you can create a knowledge base use "knowledge_base.ipynb"

response = client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='<DESCRIPTION_OF_WHEN_TO_USE_KB>',
    knowledgeBaseId='<KNOWLEDGEBASE_ID>'
)

### Step 5. Prepare agent and create agent alias

In [73]:
import time
agent_alias_name = "demo_test"

client = boto3.client("bedrock-agent", region_name=region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
response = client.prepare_agent(agentId=agent_id)
time.sleep(20)

Notice: Need to wait a minute for prepare agent

In [74]:
agent_alias_description = "init version"
# agent_alias_description = "version2"
agent_alias = client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name,
    description=agent_alias_description
)
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

### Step 6. Invoking Agent

In [75]:
# Need change to "bedrock-agent-runtime" 
client = boto3.client("bedrock-agent-runtime", region_name=region, aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

sessionid = str(uuid.uuid1())
enable_trace:bool = True 

def invoke(question: str, sessionid: str, agent_id: str, agent_alias_id: str, enable_trace=False):
    final_answer = ""
    response = client.invoke_agent(
        inputText=question, # 输入文本
        agentId=agent_id,  # 创建的 Agent 的 ID
        agentAliasId=agent_alias_id, # 创建的 Agent alias id
        sessionId=sessionid, # 当前会话的session id
        enableTrace=enable_trace # 是否打开 trace
    )
    event_stream = response['completion']
    try:
        for event in event_stream:        
            # print(event)
            if 'chunk' in event:
                data = event['chunk']['bytes']
                final_answer = data.decode('utf8')
                print(f"Final answer ->\n{final_answer}") 
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)
    # return final_answer 

##### Test
You can also test in console

In [81]:
sessionid = str(uuid.uuid1())
enable_trace:bool = True 

In [83]:
question = "Hello"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-21 07:28:41,979] p159652 {3726500921.py:23} INFO - {
  "agentId": "KX5F6ZDCMY",
  "agentAliasId": "PLBTJY8U7N",
  "sessionId": "821b355c-9fd2-11ee-b7bc-0efac58ff0bf",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "be628a3f-24e9-46a1-93ea-d9b3b91fe432-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService::generatePreviewInvoiceInfo</function_name>\n<function_description>Generate a temporary preview invoice information.</function_description>\n<required_argument>buyer_tax_

Final answer ->
Hello, I'm an invoice assistant. How may I help you with invoices today?


In [84]:
question = "I want to issue an invoice "
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-21 07:29:15,805] p159652 {3726500921.py:23} INFO - {
  "agentId": "KX5F6ZDCMY",
  "agentAliasId": "PLBTJY8U7N",
  "sessionId": "821b355c-9fd2-11ee-b7bc-0efac58ff0bf",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "28785b1b-92c5-4946-9d70-95eb0694ae0d-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService::generatePreviewInvoiceInfo</function_name>\n<function_description>Generate a temporary preview invoice information.</function_description>\n<required_argument>buyer_tax_

Final answer ->
To issue an invoice, I first need to collect some information from you. Can you please provide:

- Buyer tax number
- Buyer company name  
- Your user ID
- Product details (name, code, price for each product)


In [64]:
question = "91440300MA5FAE9E4P"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-11 21:02:52,472] p72505 {820018286.py:23} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "ZLTX6ARF1I",
  "sessionId": "5758acc8-9825-11ee-954d-4e80c71e97eb",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "87e1abf5-1a68-455a-8d7a-fedd304f17ca-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_nu

Final answer ->
Thank you for providing the buyer tax number. I still need the following invoice information from you:

What is the buyer's company name?
What is your user ID?  
What are the product details (name, code, price for each product)?


In [67]:
question = "test_company_name"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-11 21:04:45,867] p72505 {820018286.py:23} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "ZLTX6ARF1I",
  "sessionId": "5758acc8-9825-11ee-954d-4e80c71e97eb",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "d5d1184a-f4ae-44d3-89db-46b9c15f4fee-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_nu

Final answer ->
Thank you for providing all the required invoice information. I will now generate a preview invoice image for you to review before formally issuing the invoice. Please confirm after reviewing the preview image.


In [65]:
question = "00001"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-11 21:03:30,929] p72505 {820018286.py:23} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "ZLTX6ARF1I",
  "sessionId": "5758acc8-9825-11ee-954d-4e80c71e97eb",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "bb1e653a-5b73-4f5d-ac08-00eaff80dd67-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_nu

Final answer ->
Thank you for providing the user ID. I still need the following invoice information from you:

What is the buyer's company name?
What are the product details (name, code, price for each product)?


In [66]:
question = "小麦，1010101020000000000，9000"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-11 21:03:57,926] p72505 {820018286.py:23} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "ZLTX6ARF1I",
  "sessionId": "5758acc8-9825-11ee-954d-4e80c71e97eb",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "ea14ff7d-6c22-4e54-a3e9-1c2277efa84d-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_nu

Final answer ->
Thank you for providing the product detail. I still need the buyer's company name from you to generate the invoice.

What is the buyer's company name?


In [68]:
question = "yes"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-11 21:06:53,802] p72505 {820018286.py:23} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "ZLTX6ARF1I",
  "sessionId": "5758acc8-9825-11ee-954d-4e80c71e97eb",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "2fec2012-28f0-495d-8cda-a8b167041840-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_nu

Final answer ->
The invoice has been successfully issued. Do you need me to email the invoice to you? If so, please provide the email address.


In [69]:
question = "yes,xiaoqunn@amazon.com"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-11 21:08:23,636] p72505 {820018286.py:23} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "ZLTX6ARF1I",
  "sessionId": "5758acc8-9825-11ee-954d-4e80c71e97eb",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "57eb72a9-0b32-4cb3-81fe-43c2459fec5b-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_nu

Final answer ->
I have emailed the invoice to xiaoqunn@amazon.com successfully. Please let me know if you need any other invoice services.


In [87]:
sessionid = str(uuid.uuid1())
enable_trace:bool = True 

In [90]:
question = "我要开发票"
answer = invoke(question, sessionid, agent_id, agent_alias_id, enable_trace)

[2023-12-11 22:33:43,873] p72505 {820018286.py:23} INFO - {
  "agentId": "FCZI2TH2IY",
  "agentAliasId": "ZLTX6ARF1I",
  "sessionId": "3777b39c-9832-11ee-954d-4e80c71e97eb",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "5aed0174-17a6-44a3-9650-42f9b255ccf7-pre-0",
        "text": "\n\nHuman: You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<functions>\n<function>\n<function_name>POST::InvoiceService_demo::generatePreviewInvoiceImage</function_name>\n<function_description>Generate a temporary preview invoice image.</function_description>\n<required_argument>buyer_tax_nu

Final answer ->
Sorry, I don't have enough information to answer that.
